In [ ]:
# Modules
import inspect
import numpy as np
import pandas as pd
import seaborn as sns
import inspect
import warnings
import pygam

from pandas import DataFrame
from sklearn import ensemble, metrics, model_selection, preprocessing, tree
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, max_error, median_absolute_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LogisticRegression
from statsmodels.formula.api import ols
from sklearn.feature_selection import RFE
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.inspection import PartialDependenceDisplay

# Suppress all warnings
warnings.filterwarnings('ignore')

## Logistic Regression

In [ ]:
# Import the data
df_selected = pd.read_csv('Data_selection.csv', index_col=0)

##### 1.1 Initial Fitting on data set with preliminary feature selection

In [ ]:
df = df_selected

# Randomly split the data set into training and testing and deal with the imbalanced dependent variable using SMOTE
y = df['TARGET']
X = df.drop('TARGET', axis=1)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size = 0.75, shuffle = True, random_state = 480)

undersampler = RandomUnderSampler(random_state=480)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# Create a logistic regression classifier object
logreg = LogisticRegression()

# Use Recursive Feature Elimination (RFE) for stepwise variable selection
rfe = RFE(logreg)
rfe.fit(X_train_resampled, y_train_resampled)

# Fit the model using the selected features
selected_features = X_train_resampled.columns[rfe.support_]
logit_model = sm.Logit(y_train_resampled, sm.add_constant(X_train_resampled[selected_features]))
result = logit_model.fit()

# Iteratively check VIF and remove variables with high VIF values
max_vif = np.inf

while max_vif > 10:
    # Calculate VIF for the selected independent variables
    vif = pd.DataFrame()
    vif['VIF Factor'] = [variance_inflation_factor(X_train_resampled[selected_features].values, i) for i in range(X_train_resampled[selected_features].shape[1])]
    vif['features'] = selected_features

    # Remove variables with high VIF values
    high_vif_features = vif[vif['VIF Factor'] > 10]['features']
    selected_features = selected_features.drop(high_vif_features)

    # Refit the logistic regression model using the remaining independent variables
    logreg.fit(X_train_resampled[selected_features], y_train_resampled)

    # Update max_vif
    max_vif = vif['VIF Factor'].max()
    
# Calculate and display the final VIF values for the remaining independent variables
vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(X_train_resampled[selected_features].values, i) for i in range(X_train_resampled[selected_features].shape[1])]
vif['features'] = selected_features
print(vif)

logit_model = sm.Logit(y_train_resampled, sm.add_constant(X_train_resampled[selected_features]))
result = logit_model.fit()

# Iteratively remove variables with high p-values and refit the model
max_pvalue = np.inf
while max_pvalue >= 0.05:
    # Remove variables with high p-values
    high_pvalue_features = result.pvalues[result.pvalues >= 0.05].index
    selected_features = selected_features.drop(high_pvalue_features)

    # Refit the logistic regression model using the remaining independent variables
    logit_model = sm.Logit(y_train_resampled, sm.add_constant(X_train_resampled[selected_features]))
    result = logit_model.fit()

    # Update max_pvalue
    max_pvalue = result.pvalues.max()

# Display the final regression results
print(result.summary())

In [ ]:
# Make predictions on the training and test sets
y_pred_train = result.predict(sm.add_constant(X_train_resampled[selected_features]))
y_pred_test = result.predict(sm.add_constant(X_test[selected_features]))

# Compute in-sample performance metrics
accuracy_train = accuracy_score(y_train_resampled, y_pred_train.round())
precision_train = precision_score(y_train_resampled, y_pred_train.round())
recall_train = recall_score(y_train_resampled, y_pred_train.round())
f1_score_train = f1_score(y_train_resampled, y_pred_train.round())
roc_auc_train = roc_auc_score(y_train_resampled, y_pred_train.round())

# Compute out-of-sample performance metrics
accuracy_test = accuracy_score(y_test, y_pred_test.round())
precision_test = precision_score(y_test, y_pred_test.round())
recall_test = recall_score(y_test, y_pred_test.round())
f1_score_test = f1_score(y_test, y_pred_test.round())
roc_auc_test = roc_auc_score(y_train_resampled, y_pred_train.round())

# assuming you have already computed the performance metrics
data = {'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-score', 'ROC AUC'],
        'Training': [accuracy_train, precision_train, recall_train, f1_score_train, roc_auc_train],
        'Test': [accuracy_test, precision_test, recall_test, f1_score_test, roc_auc_test]}
df = pd.DataFrame(data)
df

In [ ]:
rmse_train = mean_squared_error(y_train_resampled, y_pred_train, squared = False)
mae_train = mean_absolute_error(y_train_resampled, y_pred_train)

rmse_test = mean_squared_error(y_test, y_pred_test, squared = False)
mae_test = mean_absolute_error(y_test, y_pred_test)

data = {'Metric': ['RMSE', 'MAE'],
        'Training': [rmse_train, mae_train],
        'Test': [rmse_test, mae_test]}
df = pd.DataFrame(data)
df

##### 1.2 Hyperparameter Tuning using Cross-validation on data set with preliminary feature selection

In [ ]:
# Define the hyperparameter grid
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'penalty': ['l1', 'l2', 'elasticnet', 'none'],
              'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
              'max_iter': [100, 200, 300]}

# Create a logistic regression classifier object
logreg = LogisticRegression()

# Perform hyperparameter tuning using cross-validation
grid_search = GridSearchCV(logreg, param_grid, cv=5)
grid_search.fit(X_train_resampled[selected_features], y_train_resampled)

# Display the best hyperparameters
print('Best hyperparameters:', grid_search.best_params_)

##### 1.3 Refit the model using optimal parameters found through hyperparameter tuning on data set with preliminary feature selection

In [ ]:
# Create a logistic regression classifier object with the best hyperparameters
logreg = LogisticRegression(C=0.1, max_iter=300, penalty='l2', solver='sag')

# Use Recursive Feature Elimination (RFE) for stepwise variable selection
rfe = RFE(logreg)
rfe.fit(X_train_resampled, y_train_resampled)

# Fit the model using the selected features
selected_features = X_train_resampled.columns[rfe.support_]
logit_model = sm.Logit(y_train_resampled, sm.add_constant(X_train_resampled[selected_features]))
result = logit_model.fit()

# Iteratively check VIF and remove variables with high VIF values
max_vif = np.inf

while max_vif > 10:
    # Calculate VIF for the selected independent variables
    vif = pd.DataFrame()
    vif['VIF Factor'] = [variance_inflation_factor(X_train_resampled[selected_features].values, i) for i in range(X_train_resampled[selected_features].shape[1])]
    vif['features'] = selected_features

    # Remove variables with high VIF values
    high_vif_features = vif[vif['VIF Factor'] > 10]['features']
    selected_features = selected_features.drop(high_vif_features)

    # Refit the logistic regression model using the remaining independent variables
    logreg.fit(X_train_resampled[selected_features], y_train_resampled)

    # Update max_vif
    max_vif = vif['VIF Factor'].max()
    
# Calculate and display the final VIF values for the remaining independent variables
vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(X_train_resampled[selected_features].values, i) for i in range(X_train_resampled[selected_features].shape[1])]
vif['features'] = selected_features
print(vif)

logit_model = sm.Logit(y_train_resampled, sm.add_constant(X_train_resampled[selected_features]))
result = logit_model.fit()

# Iteratively remove variables with high p-values and refit the model
max_pvalue = np.inf
while max_pvalue >= 0.05:
    # Remove variables with high p-values
    high_pvalue_features = result.pvalues[result.pvalues >= 0.05].index
    selected_features = selected_features.drop(high_pvalue_features)

    # Refit the logistic regression model using the remaining independent variables
    logit_model = sm.Logit(y_train_resampled, sm.add_constant(X_train_resampled[selected_features]))
    result = logit_model.fit()

    # Update max_pvalue
    max_pvalue = result.pvalues.max()

# Display the final regression results
print(result.summary())

In [ ]:
# Make predictions on the training and test sets
y_pred_train = result.predict(sm.add_constant(X_train_resampled[selected_features]))
y_pred_test = result.predict(sm.add_constant(X_test[selected_features]))

# Compute in-sample performance metrics
accuracy_train = accuracy_score(y_train_resampled, y_pred_train.round())
precision_train = precision_score(y_train_resampled, y_pred_train.round())
recall_train = recall_score(y_train_resampled, y_pred_train.round())
f1_score_train = f1_score(y_train_resampled, y_pred_train.round())
roc_auc_train = roc_auc_score(y_train_resampled, y_pred_train.round())

# Compute out-of-sample performance metrics
accuracy_train = accuracy_score(y_test, y_pred_test.round())
precision_train = precision_score(y_test, y_pred_test.round())
recall_train = recall_score(y_test, y_pred_test.round())
f1_score_train = f1_score(y_test, y_pred_test.round())
roc_auc_train = roc_auc_score(y_train_resampled, y_pred_train.round())

# assuming you have already computed the performance metrics
data = {'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-score', 'ROC AUC'],
        'Training': [accuracy_train, precision_train, recall_train, f1_score_train, roc_auc_train],
        'Test': [accuracy_test, precision_test, recall_test, f1_score_test, roc_auc_test]}
df = pd.DataFrame(data)
df

In [ ]:
rmse_train = mean_squared_error(y_train_resampled, y_pred_train, squared = False)
mae_train = mean_absolute_error(y_train_resampled, y_pred_train)

rmse_test = mean_squared_error(y_test, y_pred_test, squared = False)
mae_test = mean_absolute_error(y_test, y_pred_test)

data = {'Metric': ['RMSE', 'MAE'],
        'Training': [rmse_train, mae_train],
        'Test': [rmse_test, mae_test]}
df = pd.DataFrame(data)
df

In [ ]:
# Get and reshape confusion matrix data
threshold = 0.5
y_pred_test = np.where(y_pred_test > threshold, 1, 0)

matrix = confusion_matrix(y_test, y_pred_test)
matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

# Build the plot
pyplot.figure(figsize=(16,7))
sns.set(font_scale=1.4)
sns.heatmap(matrix, annot=True, annot_kws={'size':10},
            cmap=pyplot.cm.Greens, linewidths=0.2)

# Add labels to the plot
class_names = ['TARGET = 0', 'TARGET = 1']
tick_marks = np.arange(len(class_names))
tick_marks2 = tick_marks + 0.5
pyplot.xticks(tick_marks, class_names, rotation=25)
pyplot.yticks(tick_marks2, class_names, rotation=0)
pyplot.xlabel('Predicted label')
pyplot.ylabel('True label')
pyplot.title('Confusion Matrix for Random Forest Model')
pyplot.show()

In [ ]:
def check_linearity(df, target, features):
    """
    Check linearity between independent variables and the log odds assumption.
    
    :param df: DataFrame containing the data
    :param target: Name of the target variable
    :param features: List of feature names
    :return: DataFrame containing the p-values for each feature
    """
    y = df[target]
    p_values = []
    for feature in features:
        x = df[feature]
        x = sm.add_constant(x)
        model = sm.Logit(y, x)
        result = model.fit()
        p_values.append(result.pvalues[1])
    
    results_df = pd.DataFrame({'Feature': features, 'p-value': p_values})
    return results_df

results_df = check_linearity(df_selected, 'TARGET', selected_features)
results_df

In [ ]:
# Calculate the Wald statistics for each variable
wald_statistics = result.params / result.bse
wald_statistics = wald_statistics ** 2

# Get the top 10 variables with the highest Wald statistics
top_10_variables = wald_statistics.nlargest(10)

# Display the results
top_10_variables

In [ ]:
# Get the indices of top 10 features
indices = np.argsort(top_10_variables)[::-1][:10]

# Fit a logistic regression model using scikit-learn
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train_resampled.iloc[:, indices], y_train_resampled)

# Calculate partial dependence
fig, ax = pyplot.subplots(5, 2, figsize=(20, 20))
pdp_results = PartialDependenceDisplay.from_estimator(logreg, X_train_resampled.iloc[:, indices], range(10),
                                                      feature_names=X_train_resampled.columns[indices],
                                                      ax=ax.ravel())
pyplot.tight_layout()